In [2]:
# If this cell does not work try running `pip install nbformat`

import io
from IPython import nbformat

nb = nbformat.read("lab3_Frequent_Sketching.ipynb", nbformat.NO_CONVERT)
word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))

print("Word count:", word_count)

c:\users\martijn.vanmeerten\appdata\local\programs\python\python39\lib\site-packages\IPython\nbformat.py:12: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  warn("The `IPython.nbformat` package has been deprecated since IPython 4.0. "


Word count: 287


## Group Number : 33
### Name : Martijn van Meerten
### ID : 4387902

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_table('./dataset_10/capture20110818.binetflow', sep=",")
df = df[df['Label'].map(lambda x: 'Background' not in str(x))]
df.loc[df['State'].isna(),'State'] = 'NA'
state_enc = LabelEncoder()
state_enc.fit(df['State'])
df['State'] = state_enc.transform(df.State)

In [5]:
def discretize_tot_bytes(dataframe):    
    totBytesBins = [-1, 800, 1300, 1900, 10000, dataframe['TotBytes'].max()]
    totBytesLabels=[1,2,3,4,5]
    dataframe['TotBytes_disc'] = pd.cut(dataframe['TotBytes'], bins=totBytesBins, labels=totBytesLabels)
    
def discretize_state(dataframe):
    totStateBins = [-1, 20, 40, 60, 80, 100, dataframe['State'].max() + 1]
    totStateLabels=[1,2,3,4,5,6]
    dataframe['State_disc'] = pd.cut(dataframe['State'], bins=totStateBins, labels=totStateLabels)
    
def combine_discrete_values(row, key1, key2):
    return int(str(int(row[key1])) + str(int(row[key2])))

def df_combine_discretization(dataframe):
    dataframe['disc'] = dataframe.apply (lambda row: combine_discrete_values(row, 'State_disc', 'TotBytes_disc'), axis=1)
    
discretize_tot_bytes(df)
discretize_state(df)
df_combine_discretization(df)

## 2. Frequent task – 1/2 A4 (Individual)

### Implementation

In [6]:
# Calculate occurrences of n-grams
def space_saving(k, three_grams):
    # Store counters in a dictionary
    value_counter = dict()
    for three_gram in three_grams:
        # If the 3-gram is already in the counter, increment
        if three_gram in value_counter:
            value_counter[three_gram] += 1
        # Else if there is room add the 3-gram with count 1
        elif len(value_counter) < k-1:
            value_counter[three_gram] = 1
        # Else find the 3-gram with min count, delete that 3-gram
        # and assign and increment its count to the new 3-gram
        else:
            key = min(value_counter, key=value_counter.get)
            min_count = value_counter[key]
            del value_counter[key]
            value_counter[three_gram] = min_count + 1
                    
    return value_counter

### 2a. Use the SPACE SAVING algorithm

In [7]:
disc = df['disc']
# Get the 3-grams of the discretized value
three_grams = list(map(tuple, [disc[i:i+3] for i in range(len(df)-2)]))
print(disc)
print(three_grams[:10])

532        35
822        12
842        12
1021       11
1022       11
           ..
1309670    11
1309736    11
1309782    11
1309783    11
1309784    51
Name: disc, Length: 122199, dtype: int64
[(35, 12, 12), (12, 12, 11), (12, 11, 11), (11, 11, 21), (11, 21, 35), (21, 35, 34), (35, 34, 35), (34, 35, 35), (35, 35, 35), (35, 35, 35)]


In [11]:
print(len(df))
number_of_counters = [100, 300, 500]
frequencies = []
# Calculate frequencies for different numbers of counters
for i in number_of_counters:
    value_counter = space_saving(i, three_grams)
    frequencies.append(sorted(value_counter.items(), key=lambda item: item[1], reverse=True)[:10])
    
# Calculate the true frequency of the 3-grams
true_frequency = []
for ngram, _ in frequencies[0]:
    true_frequency.append(three_grams.count(ngram))
    
# Calculate the error between the approximated and true frequencies
def calculate_errors(frequencies, true_frequency):
    errors = []
    for approximation in frequencies:
        error = []
        for index, (ngram, value) in enumerate(approximation):
            error.append(value - true_frequency[index])

        errors.append(error)
    
    return errors

errors = calculate_errors(frequencies, true_frequency)
    
# Print results
for index, error in enumerate(errors):
    print(f'k = {number_of_counters[index]}')
    print(f'Frequencies: {frequencies[index]}')
    print(f'Errors: {error}\n')

print(f'True frequencies = {true_frequency}')

122199
k = 100
Frequencies: [((62, 62, 62), 51996), ((11, 11, 11), 3531), ((42, 62, 62), 3233), ((62, 62, 42), 3219), ((62, 42, 62), 3174), ((62, 32, 62), 2916), ((62, 62, 32), 2911), ((32, 62, 62), 2894), ((62, 64, 62), 2673), ((62, 62, 64), 2664)]
Errors: [10, 0, 11, 11, 11, 12, 12, 10, 11, 11]

k = 300
Frequencies: [((62, 62, 62), 51987), ((11, 11, 11), 3531), ((42, 62, 62), 3223), ((62, 62, 42), 3209), ((62, 42, 62), 3164), ((62, 32, 62), 2905), ((62, 62, 32), 2900), ((32, 62, 62), 2885), ((62, 64, 62), 2663), ((62, 62, 64), 2654)]
Errors: [1, 0, 1, 1, 1, 1, 1, 1, 1, 1]

k = 500
Frequencies: [((62, 62, 62), 51986), ((11, 11, 11), 3531), ((42, 62, 62), 3222), ((62, 62, 42), 3208), ((62, 42, 62), 3163), ((62, 32, 62), 2904), ((62, 62, 32), 2899), ((32, 62, 62), 2884), ((62, 64, 62), 2662), ((62, 62, 64), 2653)]
Errors: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

True frequencies = [51986, 3531, 3222, 3208, 3163, 2904, 2899, 2884, 2662, 2653]


### 2b. Analysis and answers to the questions

All counts found by the space saving algorithm are overestimates. The overestimate is at most $n/k$, so a higher $k$ results in a more accurate approximation. This is however a trade-off in terms of space complexity, as the memory required for the algorithm increases with $k$, the number of counters. \
\
This also follows from the results above. For $k = 100$, the algorithm overestimates by roughly 10 counts, while at $k = 500$, there are no approximation errors. The 10 most frequent 3-grams can also be found in the results above.

## 3. Sketching task – 1/2 A4 (Individual)

### 3a. COUNT-MIN sketch


In [9]:
import random
import math
a = []
b = []

# Generate random values for the hash function. Different values everytime count_min_sketch is run
def generate_random_seeds(d):
    global a
    global b
    a = random.sample(range(1, (2**61 - 2)), d)
    b = random.sample(range(0, (2**61 - 2)), d)

# Pairwise independen hash function
def pairwise_independent_hashes(ngram, w, d):
    # Combine the 3-gram into a single 6-digit integer
    combined = ''
    for x in ngram:
        combined += str(x)
    
    combinedInt = int(combined)
    hashes = []
    # Hash this according to h(x) = a*x + b % p % w
    # Gives d different hash results between 0 and w
    for i in range(d):
        hashh = ((a[i] * combinedInt + b[i]) % (2**61 - 1)) % w
        hashes.append(hashh)
    
    return hashes

def count_min_sketch(w, d, ngrams):
    generate_random_seeds(d)
    # Construct d x w table
    table = [[0 for _ in range(w)] for _ in range(d)]
    for ngram in ngrams:
        hashed_values = pairwise_independent_hashes(ngram, w, d)
        for index, value in enumerate(hashed_values):
            table[index][value] += 1
            
    return table

# Lookup frequency of an ngram
def point_lookup(ngram, table):
    d = len(table)
    w = len(table[0])
    min_count = float('inf')
    # Find the row with the minimum count for ngram and return it
    hashed_values = pairwise_independent_hashes(ngram, w, d)
    for index, value in enumerate(hashed_values):
        count = table[index][value]
        if count < min_count:
            min_count = count
    
    return min_count


ws = [50, 100, 150, 200]
ds = [10, 20, 40, 80]

frequencies = []
for w in ws:
    for d in ds:
        table = count_min_sketch(w, d, three_grams)
        frequency = []
        for ngram in set(three_grams):
            frequency.append((ngram, point_lookup(ngram, table)))
            
        frequencies.append(sorted(frequency, key=lambda item: item[1], reverse=True)[:10])
            
        
errors = calculate_errors(frequencies, true_frequency)
for index, error in enumerate(errors):
    print(f'w = {ws[math.floor(index/len(ds))]}, d = {ds[index % len(ws)]}')
    print(f'Frequencies: {frequencies[index]}')
    print(f'Errors: {error}\n')

w = 50, d = 10
Frequencies: [((62, 62, 62), 52184), ((11, 11, 11), 3769), ((62, 42, 62), 3522), ((62, 62, 42), 3428), ((42, 62, 62), 3335), ((62, 62, 32), 3080), ((32, 62, 62), 3052), ((62, 32, 62), 3033), ((62, 62, 15), 2874), ((62, 15, 62), 2870)]
Errors: [198, 238, 300, 220, 172, 176, 153, 149, 212, 217]

w = 50, d = 20
Frequencies: [((62, 62, 62), 52089), ((11, 11, 11), 3565), ((62, 62, 42), 3384), ((62, 42, 62), 3355), ((42, 62, 62), 3323), ((62, 62, 32), 3040), ((62, 32, 62), 3012), ((32, 62, 62), 3005), ((62, 62, 64), 2810), ((15, 62, 62), 2755)]
Errors: [103, 34, 162, 147, 160, 136, 113, 121, 148, 102]

w = 50, d = 40
Frequencies: [((62, 62, 62), 52098), ((11, 11, 11), 3782), ((42, 62, 62), 3330), ((62, 62, 42), 3292), ((62, 42, 62), 3276), ((62, 62, 32), 3063), ((32, 62, 62), 2995), ((62, 32, 62), 2990), ((62, 62, 64), 2805), ((62, 15, 62), 2763)]
Errors: [112, 251, 108, 84, 113, 159, 96, 106, 143, 110]

w = 50, d = 80
Frequencies: [((62, 62, 62), 52079), ((11, 11, 11), 3550),

### 3b. Analysis and answers to the questions

The hash function used is $h(x) = aX + b \mod p \mod d$. The 3-gram is combined into a 6 digit integer and the hash function maps this value to between $0$ and $d$. Two hash functions with different $a$ and $b$ are pair-wise independent and therefore suited for count-min sketch.

Count-min sketch overestimates its approximations. The overestimate is:  ${\hat {a}}_{i}\leq a_{i}+\varepsilon N$ with probability $1-\delta$, where $\varepsilon = e / w$, $\delta = 1 / e^d$ and $N$ is the stream size. So by increasing $w$ and $d$ we reduce the overestimatation factor and probability, but also increase the size and time complexity. This also shows from the results, where with $w=50$ and $d=10$, the number of errors are quite substantial, while the errors with $w=200$ and $d=80$, are around 4. The 10 most frequent 3-grams can also be found in the results above.